In [ ]:
import os


In [ ]:
%%capture
!pip install unsloth
!pip install --force-reinstall --no-chache-dir --no-deps git+https://github.com/unslothai/unsloth.github

In [ ]:
from unsloth import FastLanguageModel
import torch

In [ ]:

!pip install trl accelerate

In [ ]:
# MAKE SURE TO RUN THE INSTALLATION CELL (e.g., OP6yy890VYe3) FIRST!

import os
import json
import random
import torch
from datasets import Dataset
from transformers import TextStreamer, TrainingArguments
from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported, to_sharegpt, standardize_sharegpt, apply_chat_template
from unsloth.lora import get_peft_model # Import get_peft_model from unsloth.lora
from google.colab import drive

# Google Drive'ı bağla
drive.mount('/content/drive')

In [ ]:
# --- 1. Kütüphaneleri Yükle ve Modeli Başlat ---
# Unsloth ve diğer gerekli kütüphaneleri yükle
!pip install unsloth
!pip install trl accelerate

In [ ]:
# Model parametreleri
max_seq_length = 2048 # Daha kısa süreçler için bu değeri düşünebilirsiniz (örn. 512 veya 1024)
dtype = None # Otomatik olarak ayarlanır (bfloat16 veya float16)
load_in_4bit = True

# Modeli ve tokenizer'ı yükle
# Llama-3-8b-Instruct modeli seçildi.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
# --- 2. Veri Yükleme ve Hazırlık ---

# JSONL dosyasının yolu
dosya = r'/content/drive/My Drive/akış 1000 veri.jsonl'

# Veriyi yükle ve ShareGPT formatına uygun hale getir
all_data = []

with open(dosya, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        # JSONL dosyanızın doğrudan "messages" formatında olduğunu varsayıyorum
        # {"messages": [{"role": "system", "content": "..."}, {"role": "user", "content": "..."}, {"role": "assistant", "content": "..."}]}
        if "messages" in item and isinstance(item["messages"], list):
            all_data.append(item)
        else:
            # Eğer dosyanız {"input_text": "...", "output_text": "..."} formatındaysa,
            # bunu messages formatına dönüştürmeniz gerekir. Örnek:
            user_msg = item.get("input_text")
            assistant_msg = item.get("output_text")
            if user_msg and assistant_msg:
                # Burada bir 'system' mesajı eklemek, modelin amacını daha iyi anlamasını sağlar
                system_instruction = "Görevin, verilen adımları analiz ederek bir akış diyagramı için hazırlamaktır. Her adım bir türle eşleştirilmelidir (Süreç, Koşul, Veritabanı, Mesaj Kutusu, E-posta Gönder, Başlat, Form Göster veya Web Servisi). Ayrıca, her adımın hangi adımdan sonra geldiğini de belirlemelisin. Eğer adım türü 'Koşul' ise, 'evet-sonraki' ve 'hayır-sonraki' yollarını açıkça belirtmelisin.\n\nÇıktın kesinlikle şu formatta olmalıdır:\n\nAdım [No]: [Tür] - '[Açıklama]' sonraki:[Sonraki Adım No]\n\nveya\n\nAdım [No]: [Tür] - '[Açıklama]' evet-sonraki:[Adım No], hayır-sonraki:[Adım No]\n\nYalnızca bu formatta ve açık, doğru çıktılar üret."
                all_data.append({
                    "messages": [
                        {"role": "system", "content": system_instruction},
                        {"role": "user", "content": user_msg.strip()},
                        {"role": "assistant", "content": assistant_msg.strip()}
                    ]
                })

# Veriyi karıştır ve eğitim/test olarak böl
random.shuffle(all_data)
split_index = int(len(all_data) * 0.8)

train_data_raw = all_data[:split_index]
test_data_raw = all_data[split_index:]

print(f"✅ Toplam veri: {len(all_data)}")
print(f"📚 Eğitim verisi: {len(train_data_raw)}")
print(f"🧪 Test verisi: {len(test_data_raw)}")


In [ ]:
# Hugging Face Datasets formatına dönüştür
train_dataset = Dataset.from_list(train_data_raw)
test_dataset = Dataset.from_list(test_data_raw)

# Unsloth'un ChatML template'ini uygula
# Bu, modeli Llama-3'ün standart diyalog formatına hazırlar.
# Artık özel bir 'chat_template' tanımlamıyoruz çünkü Unsloth bunu otomatik hallediyor.
train_dataset = train_dataset.map(lambda x: {"prompt": tokenizer.apply_chat_template(x["messages"], tokenize=False)}, num_proc=os.cpu_count(), desc="Applying chat template to train dataset")
test_dataset = test_dataset.map(lambda x: {"prompt": tokenizer.apply_chat_template(x["messages"], tokenize=False)}, num_proc=os.cpu_count(), desc="Applying chat template to test dataset")

# SFTTrainer'ın beklediği "text" sütununu ayarla
# apply_chat_template sonrası "text" sütunu yerine artık "prompt" sütununu kullanıyoruz.
# Bu satırlara gerek kalmadı çünkü map fonksiyonu doğrudan "prompt" sütununu oluşturuyor.
# train_dataset = train_dataset.rename_column("text", "prompt") # 'text' sütununu 'prompt' olarak yeniden adlandırabiliriz
# test_dataset = test_dataset.rename_column("text", "prompt")

In [ ]:
# Eğer modelinize ek PEFT adaptörleri eklemek isterseniz (önerilir)
model = get_peft_model(
    model=model,
    r=16, # LoRA rank, deneyebilirsiniz
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

In [ ]:
# --- 3. Modeli Fine-tune Et ---

# Eğitim argümanları
training_args = TrainingArguments(
    per_device_train_batch_size=2, # Belleğe göre ayarlayın
    gradient_accumulation_steps=4, # per_device_train_batch_size * gradient_accumulation_steps = effective batch size
    warmup_steps=30, # Isınma adımları
    num_train_epochs=5, # Veri setinizin büyüklüğüne göre ayarlayın, overfitting'i izleyin
    learning_rate=2e-5, # Daha konservatif bir öğrenme oranı, Llama-3 için genellikle iyi
    fp16=not is_bfloat16_supported(),
    bf16=is_bfloat16_supported(),
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    save_strategy="epoch", # Her epoch sonunda kaydet
    evaluation_strategy="epoch", # Her epoch sonunda değerlendir
    load_best_model_at_end=True, # En iyi modeli yükle
    metric_for_best_model="eval_loss", # En iyi model için kullanılacak metrik
    report_to="none", # WandB vs. kullanmıyorsan "none"
)

# Trainer nesnesi
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset, # Test setini doğrulama için kullan
    dataset_text_field="prompt", # apply_chat_template sonrası "text" sütununu kullanıyoruz
    max_seq_length=max_seq_length,
    dataset_num_proc=os.cpu_count(),
    packing=False, # Kısa diziler için True olabilir, ancak şu an False kalsın
    args=training_args,
)

# Eğitimi başlat
print("\n🚀 Eğitime Başlanıyor...")
trainer_stats = trainer.train()
print("✅ Eğitim Tamamlandı!")

In [ ]:
# --- 4. Modeli Kaydet (İsteğe Bağlı) ---
# Eğitilmiş modeli daha sonra kullanmak üzere kaydet
#trainer.save_model("/content/drive/My Drive/fine_tuned_flowchart_model")
#tokenizer.save_pretrained("/content/drive/My Drive/fine_tuned_flowchart_model")
#print("\n✅ Model ve Tokenizer Drive'a kaydedildi.")

# --- 5. Test ve Çıkarım (Inference) ---
print("\n--- Test ve Çıkarım ---")

# Eğitimden sonra yeni bir model yüklemeye gerek yok, eğitilmiş 'model' objesini kullanıyoruz.

# Örnek bir kullanıcı mesajı oluştur
# Bu sistem mesajı, eğitim verinizdeki sistem mesajı ile aynı olmalı!
system_message_for_inference = "Görevin, verilen adımları analiz ederek bir akış diyagramı için hazırlamaktır. Her adım bir türle eşleştirilmelidir (Süreç, Koşul, Veritabanı, Mesaj Kutusu, E-posta Gönder, Başlat, Form Göster veya Web Servisi). Ayrıca, her adımın hangi adımdan sonra geldiğini de belirlemelisin. Eğer adım türü 'Koşul' ise, 'evet-sonraki' ve 'hayır-sonraki' yollarını açıkça belirtmelisin.\n\nÇıktın kesinlikle şu formatta olmalıdır:\n\nAdım [No]: [Tür] - '[Açıklama]' sonraki:[Sonraki Adım No]\n\nveya\n\nAdım [No]: [Tür] - '[Açıklama]' evet-sonraki:[Adım No], hayır-sonraki:[Adım No]\n\nYalnızca bu formatta ve açık, doğru çıktılar üret."

user_input_example = "1: Müşteri, sistem üzerinden sipariş oluşturur.\n2: Sipariş bilgileri veritabanına kaydedilir.\n3: Ürün stokta var mı? Evet ise ürün hazırlanır ve kargoya verilir. Hayır ise müşteriye stokta olmadığı bildirilir.\n4: Ürün hazırlanır ve kargoya teslim edilir.\n5: Hayır, stokta yok. Müşteriye ürünün stokta olmadığına dair e-posta gönderilir.\n6: Sipariş müşteriye teslim edilir ve süreç tamamlanır."

messages_for_inference = [
    {"role": "system", "content": system_message_for_inference},
    {"role": "user", "content": user_input_example}
]

# Tokenize et ve generation prompt'unu ekle
input_ids = tokenizer.apply_chat_template(
    messages_for_inference,
    tokenize=True,
    add_generation_prompt=True, # Llama-3'ün üretmeye başlaması için bu gerekli
    return_tensors="pt"
).to("cuda")

# TextStreamer ile çıktıyı akıt
text_streamer = TextStreamer(tokenizer, skip_prompt=True)

# Model ile metin üret
print("\n🤖 Model Tahmini (Örnek):")
_ = model.generate(input_ids=input_ids,
                     max_new_tokens=256, # Üretilecek maksimum token sayısı
                     pad_token_id=tokenizer.eos_token_id,
                     streamer=text_streamer)


In [ ]:
# --- 6. Metrikleri Hesapla (Test Verisi Üzerinden) ---
print("\n--- Metrik Hesaplama ---")

# Gerekli kütüphaneleri yükle (Colab oturumunda bir kez çalıştırılır)
# !pip install rouge_score
# !pip install evaluate

from evaluate import load
import numpy as np

# Metrikleri yükle
bleu_metric = load("bleu")
rouge_metric = load("rouge")

true_outputs = []
pred_outputs = []

# Test setindeki raw veriyi kullan
for item in tqdm(test_data_raw, desc="Generating predictions for test set"):
    # Her bir öğenin mesajlar listesinden kullanıcı girdisini ve asistan çıktısını al
    user_msg = next((m["content"] for m in item["messages"] if m["role"] == "user"), None)
    expected_output = next((m["content"] for m in item["messages"] if m["role"] == "assistant"), None)
    system_msg = next((m["content"] for m in item["messages"] if m["role"] == "system"), None) # Sisteminizi de alın

    if user_msg and expected_output and system_msg:
        inference_messages = [
            {"role": "system", "content": system_msg},
            {"role": "user", "content": user_msg}
        ]

        inputs = tokenizer.apply_chat_template(
            inference_messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt"
        ).to(model.device)

        with torch.no_grad(): # Tahmin yaparken gradient hesaplamayı kapat
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=256, # Üretilecek maksimum token sayısı
                do_sample=False, # Daha deterministik çıktı için False
                pad_token_id=tokenizer.eos_token_id
            )

        # Prompt kısmını atlayarak sadece modelin ürettiği cevabı decode et
        # generate fonksiyonu tüm input_ids'i döndürür, bu yüzden prompt uzunluğunu çıkarırız
        predicted_output = tokenizer.decode(generated_ids[0][len(inputs["input_ids"][0]):], skip_special_tokens=True).strip()

        true_outputs.append(expected_output.strip())
        pred_outputs.append(predicted_output.strip())

# Eğer tahmin ve gerçek çıktı listeleri boş değilse metrikleri hesapla
if len(true_outputs) > 0 and len(pred_outputs) > 0:
    # ✅ Exact Match Accuracy
    exact_matches = [int(p == t) for p, t in zip(pred_outputs, true_outputs)]
    exact_accuracy = sum(exact_matches) / len(exact_matches)

    # ✅ Relaxed Accuracy (küçük harfe çevir, boşlukları kaldır)
    def relaxed_match(pred, ref):
        return pred.lower().replace(" ", "").replace("'", "") == ref.lower().replace(" ", "").replace("'", "")

    relaxed_matches = [int(relaxed_match(p, t)) for p, t in zip(pred_outputs, true_outputs)]
    relaxed_accuracy = sum(relaxed_matches) / len(relaxed_matches)

    # ✅ BLEU ve ROUGE hesapla
    # BLEU tek referans için çift liste bekler
    bleu_score = bleu_metric.compute(predictions=pred_outputs, references=[[ref] for ref in true_outputs])
    rouge_score = rouge_metric.compute(predictions=pred_outputs, references=true_outputs)

    # ✅ F1 hesaplama (basit token bazlı)
    def compute_f1_token_based(pred, ref):
        pred_tokens = pred.lower().split()
        ref_tokens = ref.lower().split()

        if not pred_tokens and not ref_tokens:
            return 1.0 # Her ikisi de boşsa mükemmel eşleşme

        common = len(set(pred_tokens) & set(ref_tokens))
        if len(pred_tokens) == 0 or len(ref_tokens) == 0:
            return 0.0 # Birisi boşsa, diğerleri değilse eşleşme yok

        precision = common / len(pred_tokens)
        recall = common / len(ref_tokens)
        if precision + recall == 0:
            return 0.0
        return 2 * (precision * recall) / (precision + recall)

    f1_scores = [compute_f1_token_based(p, r) for p, r in zip(pred_outputs, true_outputs)]
    avg_f1 = np.mean(f1_scores)

    # ✅ Sonuçları yazdır
    print(f"🔍 Ortalama Token Bazlı F1 Skoru: {avg_f1:.4f}")
    print(f"✅ Exact Match Accuracy: {exact_accuracy:.4f}")
    print(f"🎯 Relaxed Accuracy: {relaxed_accuracy:.4f}")
    print(f"📘 BLEU: {round(bleu_score['bleu'], 4)}")
    print("📗 ROUGE:", {k: round(v, 4) for k, v in rouge_score.items()})


In [ ]:
 # Örnek tahminleri göster
    print("\n--- Örnek Tahminler ve Gerçek Çıktılar ---")
    for i in range(min(15, len(test_data_raw))): # İlk 15 veya mevcut veri sayısı kadar göster
        user_msg = next((m["content"] for m in test_data_raw[i]["messages"] if m["role"] == "user"), "N/A")
        expected_output = next((m["content"] for m in test_data_raw[i]["messages"] if m["role"] == "assistant"), "N/A")

        print(f"🔹 Input: {user_msg}")
        print(f"✅ Expected: {expected_output}")
        print(f"🤖 Predicted: {pred_outputs[i]}")
        print("---")
else:
    print("Metrik hesaplamak için yeterli test verisi bulunamadı veya tahminler boş.")


In [ ]:
import re
from collections import Counter
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import numpy as np

def parse_flowchart_step(step_text):
    """
    Verilen akış diyagramı adım metnini parse ederek yapılandırılmış bir sözlük döndürür.
    Format: Adım [No]: [Tür] - '[Açıklama]' next:[Sonraki Adım No]
    veya: Adım [No]: [Tür] - '[Açıklama]' yes-next:[Adım No], no-next:[Adım No]
    """
    parsed_data = {
        "num": None,
        "type": None,
        "description": None,
        "next": None,
        "yes_next": None,
        "no_next": None,
    }

    # Adım numarasını ve türü yakala
    match = re.match(r"Adım (\d+): (Süreç|Koşul|Veritabanı|Mesaj Kutusu|E-posta Gönder|Başlat|Form Göster|Web Servisi) - '(.*?)'", step_text)
    if match:
        parsed_data["num"] = match.group(1)
        parsed_data["type"] = match.group(2)
        parsed_data["description"] = match.group(3).strip()

        # Sonraki adım bağlantılarını yakala
        remaining_text = step_text[match.end():].strip()
        if remaining_text.startswith(","):
            remaining_text = remaining_text[1:].strip() # Baştaki virgülü kaldır

        # Koşul durumu
        if "yes-next:" in remaining_text and "no-next:" in remaining_text:
            yes_match = re.search(r"yes-next:(\d+)", remaining_text)
            no_match = re.search(r"no-next:(\d+)", remaining_text)
            if yes_match and no_match:
                parsed_data["yes_next"] = yes_match.group(1)
                parsed_data["no_next"] = no_match.group(1)
        # Tekli next durumu
        elif "next:" in remaining_text:
            next_match = re.search(r"next:(\d+|None)", remaining_text)
            if next_match:
                parsed_data["next"] = next_match.group(1)
    return parsed_data

def calculate_component_metrics(true_outputs, pred_outputs):
    """
    Her bir bileşen (adım numarası, tür, sonraki adım) için doğruluk, F1, Precision, Recall hesaplar.
    """
    num_matches = []
    type_matches = []
    next_matches = [] # next, yes_next, no_next için genel eşleşme

    # Açıklama metinleri için ayrı listeler (BLEU/ROUGE zaten bunları kullanıyor)
    true_descriptions = []
    pred_descriptions = []

    # Token tabanlı F1 için
    all_true_tokens = []
    all_pred_tokens = []

    for true_text, pred_text in zip(true_outputs, pred_outputs):
        # Her bir çok satırlı çıktıyı ayrı adımlara böl
        true_steps = true_text.split('\n')
        pred_steps = pred_text.split('\n')

        # Adım numaralarına göre eşleştirme yapabilmek için sözlüklere çevir
        true_parsed_map = {p["num"]: p for p in [parse_flowchart_step(s) for s in true_steps] if p["num"]}
        pred_parsed_map = {p["num"]: p for p in [parse_flowchart_step(s) for s in pred_steps] if p["num"]}

        for num, true_step_data in true_parsed_map.items():
            pred_step_data = pred_parsed_map.get(num)

            # 1. Adım Numarası Doğruluğu (Bu zaten eşleşme anahtarımız, ancak yine de kontrol edelim)
            num_matches.append(1 if pred_step_data and pred_step_data["num"] == true_step_data["num"] else 0)

            if pred_step_data: # Eğer bu adım için bir tahmin varsa
                # 2. Adım Türü Doğruluğu
                type_matches.append(1 if pred_step_data["type"] == true_step_data["type"] else 0)

                # 3. Sonraki Adım Bağlantı Doğruluğu
                is_next_match = 0
                if true_step_data["type"] == "Koşul":
                    if (pred_step_data["yes_next"] == true_step_data["yes_next"] and
                        pred_step_data["no_next"] == true_step_data["no_next"]):
                        is_next_match = 1
                else: # Süreç, Veritabanı vb.
                    if pred_step_data["next"] == true_step_data["next"]:
                        is_next_match = 1
                next_matches.append(is_next_match)

                # 4. Açıklama Metni İçin Liste Hazırlığı (Zaten BLEU/ROUGE'da kullanılıyor)
                true_descriptions.append(true_step_data["description"])
                pred_descriptions.append(pred_step_data["description"])

                # 5. Token Tabanlı F1 için token listeleri
                all_true_tokens.extend(str(true_step_data.get("type", "")).lower().split() + \
                                      str(true_step_data.get("description", "")).lower().split() + \
                                      str(true_step_data.get("next", "")).lower().split() + \
                                      str(true_step_data.get("yes_next", "")).lower().split() + \
                                      str(true_step_data.get("no_next", "")).lower().split())
                all_pred_tokens.extend(str(pred_step_data.get("type", "")).lower().split() + \
                                      str(pred_step_data.get("description", "")).lower().split() + \
                                      str(pred_step_data.get("next", "")).lower().split() + \
                                      str(pred_step_data.get("yes_next", "")).lower().split() + \
                                      str(pred_step_data.get("no_next", "")).lower().split())
            else: # Eğer tahmin edilen adım yoksa, hepsi yanlış sayılır
                type_matches.append(0)
                next_matches.append(0)
                # Açıklama ve token listelerine boş ekle (ya da ignore edebiliriz)
                true_descriptions.append(true_step_data["description"])
                pred_descriptions.append("") # Tahmin yoksa boş açıklama

                all_true_tokens.extend(str(true_step_data.get("type", "")).lower().split() + \
                                      str(true_step_data.get("description", "")).lower().split() + \
                                      str(true_step_data.get("next", "")).lower().split() + \
                                      str(true_step_data.get("yes_next", "")).lower().split() + \
                                      str(true_step_data.get("no_next", "")).lower().split())
                # Pred tokenları boş kalır

    metrics = {}
    if num_matches:
        metrics["step_num_accuracy"] = np.mean(num_matches)
    if type_matches:
        metrics["step_type_accuracy"] = np.mean(type_matches)
    if next_matches:
        metrics["next_link_accuracy"] = np.mean(next_matches)

    # Tüm tokenların üzerinden Precision, Recall, F1 hesaplama
    if all_true_tokens and all_pred_tokens:
        # sklearn'ın precision_recall_fscore_support'u kullanmak için etiketleme yapmalıyız
        # Basit bir token tabanlı F1 için Counter kullanmak daha kolay olabilir
        common_tokens = len(list((Counter(all_pred_tokens) & Counter(all_true_tokens)).elements()))

        if len(all_pred_tokens) == 0:
            precision = 0.0
        else:
            precision = common_tokens / len(all_pred_tokens)

        if len(all_true_tokens) == 0:
            recall = 0.0
        else:
            recall = common_tokens / len(all_true_tokens)

        if precision + recall == 0:
            f1 = 0.0
        else:
            f1 = 2 * (precision * recall) / (precision + recall)

        metrics["token_precision"] = precision
        metrics["token_recall"] = recall
        metrics["token_f1"] = f1

    return metrics, true_descriptions, pred_descriptions

# Not: BLEU ve ROUGE hesaplamaları için true_descriptions ve pred_descriptions kullanılacaktır.
# Kodunuzda bu zaten mevcuttur.

In [ ]:
# Install necessary libraries for SFTTrainer
!pip install trl accelerate